In [1]:
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load       Tokens  Owns (effective)  Host ID                               Rack 
UN  172.26.0.3  70.25 KiB  16      68.2%             81b4a457-e16a-4a02-814d-f512b8a65a9d  rack1
UN  172.26.0.2  70.26 KiB  16      70.2%             3f9ad223-fdfc-4c9b-a8cd-0750da95d497  rack1
UN  172.26.0.4  70.23 KiB  16      61.6%             35f18065-50a7-4204-ae3c-847acea9bf88  rack1



In [8]:
from cassandra.cluster import Cluster
cluster = Cluster(['p6-db-1', 'p6-db-2', 'p6-db-3'])
cass = cluster.connect()

In [9]:
cass.execute("DROP KEYSPACE IF EXISTS weather")

In [10]:
cass.execute("""
CREATE KEYSPACE IF NOT EXISTS weather
WITH REPLICATION = { 
   'class' : 'SimpleStrategy', 
   'replication_factor' : 3 
};
""")

In [19]:
cass.execute("USE weather")

In [20]:
cass.execute("CREATE TYPE station_record (tmin int, tmax int)")

InvalidRequest: Error from server: code=2200 [Invalid query] message="A user type with name 'station_record' already exists"

In [23]:
cass.execute("""
create table stations(
    id TEXT,
    name TEXT STATIC,
    date DATE,
    record weather.station_record,
    PRIMARY KEY ((id), date) 
) WITH CLUSTERING ORDER BY (date ASC)
""")

In [28]:
#q1
cass.execute("describe table weather.stations").one().create_statement

"CREATE TABLE weather.stations (\n    id text,\n    date date,\n    name text static,\n    record station_record,\n    PRIMARY KEY (id, date)\n) WITH CLUSTERING ORDER BY (date ASC)\n    AND additional_write_policy = '99p'\n    AND bloom_filter_fp_chance = 0.01\n    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}\n    AND cdc = false\n    AND comment = ''\n    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}\n    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}\n    AND memtable = 'default'\n    AND crc_check_chance = 1.0\n    AND default_time_to_live = 0\n    AND extensions = {}\n    AND gc_grace_seconds = 864000\n    AND max_index_interval = 2048\n    AND memtable_flush_period_in_ms = 0\n    AND min_index_interval = 128\n    AND read_repair = 'BLOCKING'\n    AND speculative_retry = '99p';"

In [29]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName("p6")
         .config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.4.0')
         .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions")
         .getOrCreate())

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ed41c74d-08a0-49dd-82c1-25f34b98018a;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in central
	found com.gith

In [68]:
df = spark.read.text("ghcnd-stations.txt")

In [69]:
df.take(10)

[Row(value='ACW00011604  17.1167  -61.7833   10.1    ST JOHNS COOLIDGE FLD                       '),
 Row(value='ACW00011647  17.1333  -61.7833   19.2    ST JOHNS                                    '),
 Row(value='AE000041196  25.3330   55.5170   34.0    SHARJAH INTER. AIRP            GSN     41196'),
 Row(value='AEM00041194  25.2550   55.3640   10.4    DUBAI INTL                             41194'),
 Row(value='AEM00041217  24.4330   54.6510   26.8    ABU DHABI INTL                         41217'),
 Row(value='AEM00041218  24.2620   55.6090  264.9    AL AIN INTL                            41218'),
 Row(value='AF000040930  35.3170   69.0170 3366.0    NORTH-SALANG                   GSN     40930'),
 Row(value='AFM00040938  34.2100   62.2280  977.2    HERAT                                  40938'),
 Row(value='AFM00040948  34.5660   69.2120 1791.3    KABUL INTL                             40948'),
 Row(value='AFM00040990  31.5000   65.8500 1010.0    KANDAHAR AIRPORT                      

In [70]:
from pyspark.sql.functions import col, expr, rtrim

In [71]:
df2 = df.withColumn("ID", expr("substring(value, 1, 11)")).withColumn("STATE", expr("substring(value, 39, 2)")).withColumn("NAME", rtrim(expr("substring(value, 42, 30)"))).drop("value")

In [72]:
df2.take(10)

[Row(ID='ACW00011604', STATE='  ', NAME='ST JOHNS COOLIDGE FLD'),
 Row(ID='ACW00011647', STATE='  ', NAME='ST JOHNS'),
 Row(ID='AE000041196', STATE='  ', NAME='SHARJAH INTER. AIRP'),
 Row(ID='AEM00041194', STATE='  ', NAME='DUBAI INTL'),
 Row(ID='AEM00041217', STATE='  ', NAME='ABU DHABI INTL'),
 Row(ID='AEM00041218', STATE='  ', NAME='AL AIN INTL'),
 Row(ID='AF000040930', STATE='  ', NAME='NORTH-SALANG'),
 Row(ID='AFM00040938', STATE='  ', NAME='HERAT'),
 Row(ID='AFM00040948', STATE='  ', NAME='KABUL INTL'),
 Row(ID='AFM00040990', STATE='  ', NAME='KANDAHAR AIRPORT')]

In [73]:
df2.write.saveAsTable("ID_STATE_NAME_table", mode="overwrite")
df = spark.sql("""
SELECT *
FROM ID_STATE_NAME_table
WHERE STATE = "WI"
""")

In [74]:
# count = df.collect()[0]['count']
# count

In [75]:
weather_list = df.collect()

In [76]:
for row in weather_list:
    cass.execute("""
        INSERT INTO stations (ID, NAME)
        VALUES (%s, %s)
        """,(row.ID, row.NAME))

In [77]:
cass.execute("SELECT COUNT(*) FROM weather.stations").one()[0]

1313

In [78]:
#q2
cass.execute("""
    SELECT NAME 
    FROM weather.stations 
    WHERE ID = 'USW00014837'
""").one()[0]

'MADISON DANE CO RGNL AP'